# Introduction to Pydantic for LLM Structured Outputs

Pydantic is a Python library for data validation using Python type annotations.
It's particularly useful for ensuring structured outputs from Large Language Models (LLMs).

In [21]:
import pydantic
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

print(f"Using Pydantic version: {pydantic.__version__}\n")

Using Pydantic version: 2.11.4



In [23]:
# Define a User model
class User(BaseModel):
    id: int  
    name: str

In [24]:
# Create a valid user
user1 = User(id=1, name="John Doe")
print(user1)

id=1 name='John Doe'


In [28]:
# Create an invalid user
# user2 = User(id="a", name="John Doe")
# print(user2)

## Part 1: Common Data Types in Pydantic

### String Types

In [35]:
class StringExample(BaseModel):
    name: str
    description: Optional[str] = None  # Optional field with default None
    email: str = Field(..., pattern=r"^\S+@\S+\.\S+$")  # With validation regex

In [37]:
# Demonstration
print("\nString Type Example:")
valid_string = StringExample(
    name="John Doe", email="john@example.com", description="A sample user"
)
print(f"Valid data: {valid_string.model_dump_json(indent=2)}")

try:
    invalid_string = StringExample(
        name="Jane Doe",
        email="not-an-email",  # Invalid email format
    )
except Exception as e:
    print(f"Invalid data error: {e}")


String Type Example:
Valid data: {
  "name": "John Doe",
  "description": "A sample user",
  "email": "john@example.com"
}
Invalid data error: 2 validation errors for StringExample
description
  Field required [type=missing, input_value={'name': 'Jane Doe', 'email': 'not-an-email'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
email
  String should match pattern '^\S+@\S+\.\S+$' [type=string_pattern_mismatch, input_value='not-an-email', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_pattern_mismatch


### Numeric Types

In [38]:
class NumericExample(BaseModel):
    count: int
    price: float
    # Constrained numeric types:
    probability: float = Field(..., ge=0.0, le=1.0)  # Between 0 and 1
    age: int = Field(..., gt=0, lt=120)  # Between 1 and 119

In [39]:
# Demonstration
print("\nNumeric Type Example:")
valid_numeric = NumericExample(count=5, price=19.99, probability=0.75, age=30)
print(f"Valid data: {valid_numeric.model_dump_json(indent=2)}")

try:
    invalid_numeric = NumericExample(
        count=5,
        price=19.99,
        probability=1.5,  # Outside allowed range
        age=30,
    )
except Exception as e:
    print(f"Invalid data error: {e}")


Numeric Type Example:
Valid data: {
  "count": 5,
  "price": 19.99,
  "probability": 0.75,
  "age": 30
}
Invalid data error: 1 validation error for NumericExample
probability
  Input should be less than or equal to 1 [type=less_than_equal, input_value=1.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal


### Boolean Types

In [40]:
class BooleanExample(BaseModel):
    is_active: bool
    has_subscription: bool = False  # With default value

In [41]:
# Demonstration
print("\nBoolean Type Example:")
valid_boolean = BooleanExample(is_active=True)
print(f"Valid data: {valid_boolean.model_dump_json(indent=2)}")

try:
    invalid_boolean = BooleanExample(is_active="yes")  # Not a boolean
except Exception as e:
    print(f"Invalid data error: {e}")


Boolean Type Example:
Valid data: {
  "is_active": true,
  "has_subscription": false
}


### Literal Types

In [42]:
class LiteralExample(BaseModel):
    status: Literal["pending", "approved", "rejected"]
    role: Literal["admin", "user", "guest"] = "user"  # With default


In [43]:
# Demonstration
print("\nLiteral Type Example:")
valid_literal = LiteralExample(status="approved")
print(f"Valid data: {valid_literal.model_dump_json(indent=2)}")

try:
    invalid_literal = LiteralExample(status="waiting")  # Not in allowed literals
except Exception as e:
    print(f"Invalid data error: {e}")


Literal Type Example:
Valid data: {
  "status": "approved",
  "role": "user"
}
Invalid data error: 1 validation error for LiteralExample
status
  Input should be 'pending', 'approved' or 'rejected' [type=literal_error, input_value='waiting', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error


### List Types

In [44]:
class ListExample(BaseModel):
    tags: List[str]

In [45]:
# Demonstration
print("\nList Type Example:")
valid_list = ListExample(
    tags=["python", "data", "validation"]
)
print(f"Valid data: {valid_list.model_dump_json(indent=2)}")

try:
    invalid_list = ListExample(
        tags=["python", "data", "validation"],
    )
except Exception as e:
    print(f"Invalid data error: {e}")


List Type Example:
Valid data: {
  "tags": [
    "python",
    "data",
    "validation"
  ]
}


## Part 2: Example Data Models for LLM Structured Outputs

### Example 1: Customer Sentiment Analysis

In [46]:
class SentimentAnalysis(BaseModel):
    text: str
    sentiment: Literal["positive", "negative", "neutral"]
    confidence: float = Field(..., gt=0.0, le=1.0)
    key_phrases: List[str] = Field(..., min_length=2)

In [47]:
# Test example
sentiment_analysis = SentimentAnalysis(
    text="I really enjoyed using this product. It's fantastic!",
    sentiment="positive",
    confidence=0.92,
    key_phrases=["enjoyed", "fantastic"],
)
print(f"\nValid SentimentAnalysis:\n{sentiment_analysis.model_dump_json(indent=2)}")


Valid SentimentAnalysis:
{
  "text": "I really enjoyed using this product. It's fantastic!",
  "sentiment": "positive",
  "confidence": 0.92,
  "key_phrases": [
    "enjoyed",
    "fantastic"
  ]
}


### Example 2: Product Recommendation

In [48]:
class Product(BaseModel):
    name: str
    description: str
    price: float = Field(..., gt=0)
    category: str
    in_stock: bool


class ProductRecommendation(BaseModel):
    user_query: str
    recommended_products: List[Product]
    reasoning: str
    personalization_level: float = Field(..., ge=0.0, le=1.0)

In [51]:
product_recommendation = ProductRecommendation(
    user_query="I need a laptop for gaming under $2000",
    recommended_products=[
        Product(
            name="GamerBook Pro",
            description="High-performance gaming laptop with RGB keyboard",
            price=1899.99,
            category="Electronics",
            in_stock=True,
        ),
        Product(
            name="PowerGamer X",
            description="Affordable gaming laptop with dedicated GPU",
            price=1499.99,
            category="Electronics",
            in_stock=False,
        ),
    ],
    reasoning="Selected based on gaming requirements and budget constraints",
    personalization_level=0.85,
)
print(
    f"\nValid ProductRecommendation:\n{product_recommendation.model_dump_json(indent=2)}"
)


Valid ProductRecommendation:
{
  "user_query": "I need a laptop for gaming under $2000",
  "recommended_products": [
    {
      "name": "GamerBook Pro",
      "description": "High-performance gaming laptop with RGB keyboard",
      "price": 1899.99,
      "category": "Electronics",
      "in_stock": true
    },
    {
      "name": "PowerGamer X",
      "description": "Affordable gaming laptop with dedicated GPU",
      "price": 1499.99,
      "category": "Electronics",
      "in_stock": false
    }
  ],
  "reasoning": "Selected based on gaming requirements and budget constraints",
  "personalization_level": 0.85
}


### Example 3: Content Generation with Metadata

In [52]:
class ContentType(BaseModel):
    title: str
    content: str
    word_count: int = Field(..., gt=0)
    tags: List[str] = Field(..., min_length=1, max_length=10)
    target_audience: Literal["general", "technical", "business", "academic"]
    reading_time_minutes: int = Field(..., gt=0)
    seo_score: float = Field(..., ge=0.0, le=1.0)
    contains_code_snippets: bool = False

In [53]:
content = ContentType(
    title="Getting Started with Pydantic",
    content="Pydantic is a powerful library for data validation...",
    word_count=1200,
    tags=["python", "data-validation", "pydantic", "tutorial"],
    target_audience="technical",
    reading_time_minutes=6,
    seo_score=0.87,
    contains_code_snippets=True,
)
print(f"\nValid ContentType:\n{content.model_dump_json(indent=2)}")


Valid ContentType:
{
  "title": "Getting Started with Pydantic",
  "content": "Pydantic is a powerful library for data validation...",
  "word_count": 1200,
  "tags": [
    "python",
    "data-validation",
    "pydantic",
    "tutorial"
  ],
  "target_audience": "technical",
  "reading_time_minutes": 6,
  "seo_score": 0.87,
  "contains_code_snippets": true
}
